In [1]:
import numpy as np
import ROOT as rt

Welcome to JupyROOT 6.10/08


Efficiency calculator

In [2]:
def compute_Eff(p, t, alpha = 0.05):
    e = p/t
    if e == 0:
        de = 1 - np.power(alpha, 1/t)
    else:
        de = np.sqrt(p*(1-e))/t
    return e,de

print compute_Eff(2000., 45000.)

print compute_Eff(0, 1.E9)

(0.044444444444444446, 0.0009714723739706669)
(0.0, 2.9957322178475465e-09)


# Analysis

In [3]:
N_bkg = 3100
N_sig = 200

binning = [40, 0, 400]

histo_bkg = {'path':'../_root/histos/RecoMassSpectrum_bkg.root', 'name':'h_mass_200-INF'}

In [7]:
w = rt.RooWorkspace()
w.SetNameTitle('w_TOF_HSCP', 'w_TOF_HSCP')
w.addClassDeclImportDir("/Users/olmo/cernbox/PID_timing_studies/script_analysis")
w.importClassCode("RooGausDoubleExp", 1)


w.factory('Exponential:bkg_pdf(Mass[{},{}], lambda[-0.023,-5,-0.001])'.format(binning[1], binning[2]))
w.factory("GausDoubleExp:sig_pdf(Mass, mu[200., 100, 4000], sigma[30., 10, 500], aL[2, 0.1, 3], aR[1.5, 0.1, 3])");
w.factory('SUM:model(nsig[0,10000]*sig_pdf, nbkg[0,10000]*bkg_pdf)')

w.var('Mass').setBins(binning[0])

[#0] ERROR:ObjectHandling -- RooFactoryWSTool::createArg() ERROR class GausDoubleExp not defined in ROOT class table
[#0] ERROR:ObjectHandling -- RooFactoryWSTool::createArg() ERROR class GausDoubleExp not found in factory alias table, nor in ROOT class table
[#0] ERROR:ObjectHandling -- RooFactoryWSTool::processExpression() ERRORS detected, transaction to workspace aborted, no objects committed
[#0] ERROR:ObjectHandling -- RooFactoryWSTool::add(model) ERROR creating RooAddPdf: RooAbsPdf named sig_pdf not found
[#0] ERROR:ObjectHandling -- RooFactoryWSTool::processExpression() ERRORS detected, transaction to workspace aborted, no objects committed


In [ ]:
f_bkg = rt.TFile.Open(histo_bkg['path'])
h_bkg = f_bkg.Get(histo_bkg['name'])
h_bkg.SetNameTitle('h_bkg', 'h_bkg')

roohist_bkg = rt.RooDataHist('bkg_MCHist_pdf', 'bkg_MCHist_pdf', rt.RooArgList(w.var('Mass')), h_bkg)
bkg_histpdf = rt.RooHistPdf('bkg_MCHist_pdf', 'bkg_MCHist_pdf', rt.RooArgSet(w.var('Mass')), roohist_bkg, 2)

# getattr(w,"import")(bkg_histpdf, 'bkg_MCHist_pdf', 1)
# w.factory('AbsPdf:bkg_MCHist_abspdf(Mass,bkg_MCHist_pdf)')

getattr(w,"import")(bkg_histpdf)
w.Print()


w.factory('Gaussian:sig_data_pdf(Mass, mu_data[200.], sigma_data[30.])')
w.factory('SUM:data_pdf(nsig_data[0,10000]*sig_data_pdf, nbkg_data[0,10000]*bkg_MCHist_pdf)')


w.Print()
# w.factory('DataHist:bkg_MCHist(Mass, h_bkg)')
# w.factory('HistPdf:bkg_MCHist_pdf(Mass,h_bkg)')

In [ ]:
pdf = w.pdf("model")
rt.RooRealVar *m = w.var("Mass")
# m.setBins(binning[0])

w.var("nsig").setVal(N_sig)
w.var("nbkg").setVal(N_bkg)

In [ ]:
data = pdf.generate(m)

data.SetName("data")
# w.import(data)

data.Print()

In [ ]:
import ROOT
from ROOT import RooFit as RF
from ROOT import RooStats as RS

NTOYS = 1000


# ROOT settings
ROOT.Math.MinimizerOptions.SetDefaultMinimizer('Minuit2')

# Workspace
w = ROOT.RooWorkspace('w')

# Observable
E = w.factory('E[0.,100.]')

# Constrained parameters and constraint PDF
mean = w.factory('mean[50.,49.,51.]')
mean_obs = w.factory('mean_obs[50.,49.,51.]')
mean_obs.setConstant(True)
mean_err = w.factory('mean_err[0.2]')
cpdf_mean = w.factory('Gaussian::cpdf_mean(mean,mean_obs,mean_err)')

sigma = w.factory('sigma[1.,0.5,1.5]')
sigma_obs = w.factory('sigma_obs[1.,0.5,1.5]')
sigma_obs.setConstant(True)
sigma_err = w.factory('sigma_err[0.1]')
cpdf_sigma = w.factory('Gaussian::cpdf_sigma(sigma,sigma_obs,sigma_err)')

# Signal
n_sig = w.factory('n_sig[0.,0.,10.]')
pdf_sig = w.factory('Gaussian::pdf_sig(E,mean,sigma)')

# Background
n_bkg = w.factory('n_bkg[10.,0.,50.]')
pdf_bkg = w.factory('Polynomial::pdf_bkg(E,{})')

# PDF
pdf_sum = w.factory('SUM::pdf_sum(n_sig*pdf_sig,n_bkg*pdf_bkg)')
pdf_const = w.factory('PROD::pdf_const({pdf_sum,cpdf_mean,cpdf_sigma})')

# ModelConfig
mc = RS.ModelConfig('mc', w)
mc.SetPdf( pdf_const )
mc.SetParametersOfInterest( ROOT.RooArgSet(n_sig) )
mc.SetObservables( ROOT.RooArgSet(E) )
mc.SetConstraintParameters( ROOT.RooArgSet(mean, sigma) )
mc.SetNuisanceParameters( ROOT.RooArgSet(mean, sigma, n_bkg) )
mc.SetGlobalObservables( ROOT.RooArgSet(mean_obs, sigma_obs) )

# Create toy data
data = pdf_sum.generate(ROOT.RooArgSet(E), RF.Name('data'), RF.Verbose(True), RF.Extended())
ROOT.RooAbsData.setDefaultStorageType(ROOT.RooAbsData.Vector)
data.convertToVectorStore()

# Print workspace
w.Print()

# Get RooStats ModelConfig from workspace and save it as Signal+Background model
sbModel = mc
poi = sbModel.GetParametersOfInterest().first()
poi.setVal(1.)
sbModel.SetSnapshot(ROOT.RooArgSet(poi))

# Clone S+B model, set POI to zero and set as B-Only model
bModel = mc.Clone()
bModel.SetName('mc_with_poi_0')
poi.setVal(0.)
bModel.SetSnapshot(ROOT.RooArgSet(poi))

RS.UseNLLOffset(True)

hc = RS.FrequentistCalculator(data, bModel, sbModel)
hc.SetToys(int(NTOYS), int(NTOYS/2.))
hc.StoreFitInfo(True)
hc.UseSameAltToys()

# Test statistics a: profile likelihood
profll = RS.ProfileLikelihoodTestStat(sbModel.GetPdf())
profll.EnableDetailedOutput()
profll.SetLOffset(True)
profll.SetMinimizer('Minuit2')
profll.SetOneSided(False)
profll.SetPrintLevel(0)
profll.SetStrategy(2)
profll.SetAlwaysReuseNLL(True)
profll.SetReuseNLL(True)

# Test statistics b
eventstat = RS.NumEventsTestStat(sbModel.GetPdf())

# Choose test statistics
teststat = profll
# teststat = eventstat

toymcs = hc.GetTestStatSampler()
toymcs.SetTestStatistic(teststat)
toymcs.SetUseMultiGen(True)

# HypoTestInverter
calc = RS.HypoTestInverter(hc)
calc.SetConfidenceLevel(0.90)
calc.UseCLs(True)
calc.SetVerbose(False)
calc.SetFixedScan(6, 0., 5., False)
hypotestresult = calc.GetInterval()

# Plot HypoTestInverter result
canvas = ROOT.TCanvas()
plot = RS.HypoTestInverterPlot('hypotest_inverter', 'hypotest inverter', hypotestresult)
plot.MakePlot()
plot.MakeExpectedPlot()
plot.Draw('CLB2CL')

# Print information
hc.GetFitInfo().Print('v')
teststat.GetDetailedOutput().Print('v')